In [4]:
import os
import chromadb
from chromadb.utils import embedding_functions
import re
import pathlib

class ChromaVectorDB:
    """ChromaDB를 이용한 벡터 DB 관리 클래스"""
    
    def __init__(self, embedding_model, persist_directory="./chroma_db", collection_name="Gray"):
        """
        ChromaDB 초기화
        
        Args:
            embedding_model: 임베딩 모델 (SentenceTransformer 또는 호환 래퍼)
            persist_directory (str): 벡터 DB가 저장될 디렉토리 경로
            collection_name (str): 컬렉션 이름
        """
        # 디렉토리 생성 확인
        os.makedirs(persist_directory, exist_ok=True)
        
        # 임베딩 모델 저장
        self.embedding_model = embedding_model
        
        # 기존 데이터베이스 디렉토리 확인 및 처리
        db_path = pathlib.Path(persist_directory)
        db_exists = db_path.exists() and any(db_path.iterdir())
        
        # 사용자 정의 임베딩 함수 생성 - 기존 embedding_model을 활용
        self.embedding_function = CustomEmbeddingFunction(self.embedding_model)
        
        # 만약 데이터베이스가 손상되었거나 호환성 문제가 있다면 다시 생성
        try:
            self.client = chromadb.PersistentClient(path=persist_directory)
            
            # 컬렉션 가져오기
            try:
                self.collection = self.client.get_collection(
                    name=collection_name,
                    embedding_function=self.embedding_function
                )
                print(f"기존 컬렉션 '{collection_name}'을 로드했습니다.")
            except Exception as e:
                print(f"컬렉션 로드 중 오류 발생: {e}")
                
                # 오류가 'max_seq_id' 관련 문제인지 확인
                if "max_seq_id" in str(e) or "PersistentData" in str(e):
                    print("데이터베이스 구조 호환성 문제가 발견되었습니다. 컬렉션을 재생성합니다.")
                    
                    # 기존 컬렉션 제거 시도
                    try:
                        self.client.delete_collection(name=collection_name)
                        print(f"기존 컬렉션 '{collection_name}'을 삭제했습니다.")
                    except:
                        pass
                    
                    # 새 컬렉션 생성
                    self.collection = self.client.create_collection(
                        name=collection_name,
                        embedding_function=self.embedding_function
                    )
                    print(f"새 컬렉션 '{collection_name}'을 생성했습니다.")
                else:
                    # 다른 종류의 오류면 새 컬렉션 생성
                    print(f"새 컬렉션 '{collection_name}'을 생성합니다.")
                    self.collection = self.client.create_collection(
                        name=collection_name,
                        embedding_function=self.embedding_function
                    )
                    
        except Exception as outer_e:
            print(f"치명적 오류: ChromaDB 클라이언트 초기화 실패: {outer_e}")
            print("백업 솔루션: 기존 데이터베이스를 재설정합니다.")
            
            # 데이터베이스 디렉토리가 존재하면 백업 후 재생성
            if db_exists:
                # 기존 폴더 백업 (이름 변경)
                import shutil
                import time
                
                backup_dir = f"{persist_directory}_backup_{int(time.time())}"
                try:
                    shutil.move(persist_directory, backup_dir)
                    print(f"기존 데이터베이스를 {backup_dir}로 백업했습니다.")
                except Exception as move_err:
                    print(f"백업 실패: {move_err}")
                    # 기존 폴더 삭제 시도
                    try:
                        shutil.rmtree(persist_directory)
                        print(f"기존 데이터베이스 폴더를 삭제했습니다.")
                    except Exception as rm_err:
                        print(f"폴더 삭제 실패: {rm_err}")
            
            # 폴더 재생성
            os.makedirs(persist_directory, exist_ok=True)
            
            # 클라이언트와 컬렉션 새로 생성
            self.client = chromadb.PersistentClient(path=persist_directory)
            self.collection = self.client.create_collection(
                name=collection_name,
                embedding_function=self.embedding_function
            )
            print(f"ChromaDB를 재설정하고 새 컬렉션 '{collection_name}'을 생성했습니다.")
    
    def change_collection(self, collection_name):
        """컬렉션 변경"""
        try:
            self.collection = self.client.get_collection(name=collection_name)
            self.collection_name = collection_name
            return True
        except Exception as e:
            print(f"컬렉션 변경 실패: {e}")
            return False
        
    def search_with_embedding(self, embedding, top_n=5):
        """
        임베딩 벡터를 사용하여 검색
        
        Args:
            embedding (list): 쿼리 임베딩 벡터
            top_n (int): 반환할 결과 수
            metadata_filter (dict, optional): 메타데이터 필터링 조건
        
        Returns:
            dict: 검색 결과
        """
        try :
            results = self.collection.query(
                query_embeddings=[embedding],
                n_results=top_n,
                include=["documents", "metadatas", "distances"]
            )
            
            # 결과 포맷팅
            formatted_results = []
            if results['documents'] and results['documents'][0]:
                for i, (doc, metadata, distance) in enumerate(zip(
                    results['documents'][0], 
                    results['metadatas'][0], 
                    results['distances'][0]
                )):
                    formatted_results.append({
                        'id': results['ids'][0][i] if 'ids' in results else f"doc_{i}",
                        'document': doc,
                        'chunk': doc,
                        'distance': distance,
                        'page': metadata.get('page', 1) if metadata else 1,
                        'index': i
                    })
            
            return formatted_results
        except Exception as e:
            print(f"임베딩 검색 실패: {e}")
            return []
    
    def add_documents(self, documents, metadatas=None, ids=None):
        """
        문서를 벡터 DB에 추가
        
        Args:
            documents (list): 문서 텍스트 리스트
            metadatas (list, optional): 각 문서에 대한 메타데이터 리스트
            ids (list, optional): 각 문서에 대한 고유 ID 리스트
        
        Returns:
            int: 추가된 문서 수
        """
        if ids is None:
            # 고유한 ID 생성 (timestamp + index)
            import time
            timestamp = int(time.time())
            ids = [f"doc_{timestamp}_{i}" for i in range(len(documents))]
        
        # 메타데이터가 제공되지 않은 경우 빈 딕셔너리 생성
        if metadatas is None:
            metadatas = [{} for _ in range(len(documents))]
        
        # 문서 임베딩 미리 계산 - 디버깅 용도
        # embeddings = self.embedding_model.encode(documents)
        
        # 문서 추가 (임베딩은 임베딩 함수가 자동 계산)
        self.collection.add(
            documents=documents,
            metadatas=metadatas,
            ids=ids
        )
        
        return len(documents)
    
    def load_document_file(self, file_path, chunk_size=None):
        """
        파일을 로드하여 벡터 DB에 저장
        
        Args:
            file_path (str): 로드할 파일 경로
            chunk_size (int, optional): 청크 분할 크기 (None이면 페이지 단위로 분할)
            
        Returns:
            int: 추가된 청크 수
        """
        # 파일 확장자 확인
        # _, ext = os.path.splitext(file_path)
        
        # 파일 읽기
        try:
            print(f"파일 로드 중: {file_path}")
            text_content = pathlib.Path(file_path).read_text(encoding="utf-8")
            print(f"파일 크기: {len(text_content)} 문자")
        except Exception as e:
            print(f"파일 로드 오류: {e}")
            return 0
        
        # 청크 분할 방식 결정
        chunks = []
        metadatas = []
        
        if chunk_size:
            # 지정된 청크 크기로 분할
            current_pos = 0
            while current_pos < len(text_content):
                chunk = text_content[current_pos:current_pos + chunk_size]
                chunks.append(chunk)
                metadatas.append({"source": file_path, "chunk_index": len(chunks)})
                current_pos += chunk_size
            
            print(f"크기 기반 분할: {len(chunks)}개 청크 생성")
        else:
            # 페이지 단위 분할 (#### Page X 패턴 사용)
            page_pattern = r'(####\s+Page\s+\d+\s*\n(?:[\s\S]*?)(?=####\s+Page\s+\d+\s*\n|$))'
            page_chunks = re.findall(page_pattern, text_content)
            
            if page_chunks:
                # 페이지 패턴 찾음
                for page_chunk in page_chunks:
                    # 페이지 번호 추출
                    page_match = re.match(r'####\s+Page\s+(\d+)', page_chunk)
                    page_num = page_match.group(1) if page_match else "unknown"
                    
                    chunks.append(page_chunk)
                    metadatas.append({"source": file_path, "page": f"Page {page_num}"})
                
                print(f"페이지 기반 분할: {len(chunks)}개 페이지 찾음")
            else:
                # 페이지 패턴 못찾음 - 문단 단위로 분할
                paragraphs = re.split(r'\n\s*\n', text_content)
                paragraphs = [p.strip() for p in paragraphs if p.strip()]
                
                for i, para in enumerate(paragraphs):
                    chunks.append(para)
                    metadatas.append({"source": file_path, "paragraph": i+1})
                
                print(f"문단 기반 분할: {len(chunks)}개 문단 생성")
        
        # 청크가 없으면 전체 텍스트를 하나의 청크로 처리
        if not chunks:
            chunks = [text_content]
            metadatas = [{"source": file_path, "full_document": True}]
            print("분할 실패: 전체 텍스트를 하나의 청크로 처리")
        
        # 벡터 DB에 추가
        added_count = self.add_documents(
            documents=chunks,
            metadatas=metadatas
        )
        
        print(f"벡터 DB에 {added_count}개 청크 추가 완료")
        return added_count
    
    # def search(self, query=None, query_embeddings=None, top_n=5, metadata_filter=None):
    #     """
    #     벡터 유사도 기반 검색 수행
        
    #     Args:
    #         query (str, optional): 텍스트 검색 쿼리 (query_embeddings이 제공되지 않을 때 사용)
    #         query_embeddings (list, optional): 직접 제공하는 쿼리 임베딩 리스트
    #         top_n (int): 반환할 결과 수
    #         metadata_filter (dict, optional): 메타데이터 필터링 조건
        
    #     Returns:
    #         dict: 검색 결과
    #     """
    #     # 쿼리 임베딩 처리
    #     if query_embeddings is None and query is not None:
    #         query_embedding = self.embedding_model.encode([query])[0]
    #         query_embeddings = [query_embedding.tolist()]
    #         print(f"쿼리 임베딩 shape: {query_embedding.shape}")
        
    #     # ChromaDB를 통한 검색
    #     results = self.collection.query(
    #         query_embeddings=query_embeddings,  # query_texts 대신 query_embeddings 사용
    #         n_results=top_n,
    #         where=metadata_filter,
    #         include=["documents", "metadatas", "distances", "embeddings"]
    #     )
        
    #     # 결과 포맷팅
    #     formatted_results = []
    #     for i in range(len(results["documents"][0])):
    #         formatted_results.append({
    #             "chunk": results["documents"][0][i],
    #             "id": results["ids"][0][i],
    #             "metadata": results["metadatas"][0][i],
    #             "distance": results["distances"][0][i] if "distances" in results else None,
    #             # "embedding": results["embeddings"][0][i] if "embeddings" in results else None
    #         })
        
    #     return formatted_results

    # def search(self, query, top_n=5, metadata_filter=None):
    #     """
    #     벡터 유사도 기반 검색 수행
        
    #     Args:
    #         query (str): 검색 쿼리
    #         top_n (int): 반환할 결과 수
    #         metadata_filter (dict, optional): 메타데이터 필터링 조건
        
    #     Returns:
    #         dict: 검색 결과
    #     """
    #     # 진단용 - 쿼리 임베딩 계산
    #     query_embedding = self.embedding_model.encode([query])[0]
    #     print(f"쿼리 임베딩 shape: {query_embedding.shape}")
        
    #     # ChromaDB를 통한 검색
    #     results = self.collection.query(
    #         query_texts=[query],
    #         n_results=top_n,
    #         where=metadata_filter,
    #         include=["documents", "metadatas", "distances", "embeddings"]
    #     )
        
    #     # 결과 포맷팅
    #     formatted_results = []
    #     for i in range(len(results["documents"][0])):
    #         formatted_results.append({
    #             "chunk": results["documents"][0][i],
    #             "id": results["ids"][0][i],
    #             "metadata": results["metadatas"][0][i],
    #             "distance": results["distances"][0][i] if "distances" in results else None,
    #             # "embedding": results["embeddings"][0][i] if "embeddings" in results else None
    #         })
        
    #     return formatted_results
    
    # def get_all_document_embeddings(self, ids=None):
    #     """
    #     저장된 모든 문서의 임베딩 조회 (디버깅 용도)
    #     """
    #     result = self.collection.get(ids=ids, include=["embeddings", "documents"])
    #     return result
    
    # def get_document_by_id(self, doc_id):
    #     """ID로 문서 조회"""
    #     result = self.collection.get(ids=[doc_id])
    #     if result["documents"]:
    #         return {
    #             "chunk": result["documents"][0],
    #             "metadata": result["metadatas"][0]
    #         }
    #     return None
    
    # def delete_document(self, doc_id):
    #     """ID로 문서 삭제"""
    #     self.collection.delete(ids=[doc_id])
    
    def delete_collection(self):
        """컬렉션 삭제"""
        self.client.delete_collection(self.collection.name)
    
    def get_collection_stats(self):
        """컬렉션 통계 조회"""
        count = self.collection.count()
        return {
            "document_count": count,
            "collection_name": self.collection.name
        }
    
class CustomEmbeddingFunction(embedding_functions.EmbeddingFunction):
    """기존 임베딩 모델을 ChromaDB에서 사용하기 위한 래퍼 클래스"""
    
    def __init__(self, embedding_model):
        """
        생성자
        
        Args:
            embedding_model: 임베딩 모델 (encode 메서드 제공)
        """
        self.embedding_model = embedding_model
    
    def __call__(self, texts):
        """
        텍스트를 임베딩 벡터로 변환
        
        Args:
            texts: 인코딩할 텍스트 리스트
        
        Returns:
            임베딩 벡터 리스트
        """
        return self.embedding_model.encode(texts).tolist()

In [5]:
import numpy as np
import re
import os
import json
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from model_loader.config import embedding_loader, generation_loader

class HybridSearcher:
    def __init__(self, embedding_model, chunk_size=200, chunk_overlap=50, 
                 persist_directory="./chroma_db", collection_name="Gray"):
        """
        하이브리드 검색 클래스 초기화
        
        Args:
            embedding_model: 임베딩 모델
            chunk_size (int): 청크 크기
            chunk_overlap (int): 청크 간 중복 크기
            persist_directory (str): 벡터 DB 저장 경로
            collection_name (str): 벡터 DB 컬렉션 이름
        """
        self.embedding_model = embedding_model
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.collection_name = collection_name
        self.chunks = None
        self.chunk_metadata = None
        self.bm25_index = None
        self.vector_index = None
        self.persist_directory=persist_directory
        
        # ChromaDB 벡터 DB 클래스 초기화 - 동일한 임베딩 모델 사용
        self.vector_db = ChromaVectorDB(
            embedding_model=self.embedding_model,
            persist_directory=persist_directory,
            collection_name=collection_name
        )
        
        # 검색 결과 비교용 플래그
        self.debug_mode = False
        
        # 프롬프트 로드
        self.complexity_prompt = self._load_prompt("prompts/en/complex/complex_prompt.txt")
        self.decompose_prompt = self._load_prompt("prompts/en/decompose/decompose_prompt.txt")

    def _load_prompt(self, file_path):
        """프롬프트 파일 로드"""
        try:
            with open(file_path, "r", encoding="utf-8") as f:
                return f.read()
        except Exception as e:
            print(f"프롬프트 파일 로드 중 오류 발생: {e}")
            # 기본 프롬프트 반환
            if "complex" in file_path:
                return "질문을 분석하여 복합적인 질문인지 판단하세요. 복합적인 질문은 여러 하위 질문으로 분해할 수 있습니다. '예' 또는 '아니오'로만 답변하세요."
            elif "decompose" in file_path:
                return "다음 복합적인 질문을 여러 개의 간단한 하위 질문으로 분해하세요. JSON 형식으로 하위 질문 목록을 반환하세요."

    def multi_collection_search(self, query, top_n_per_collection=5, final_top_n=5, alpha=0.5):
        """
        여러 컬렉션에서 검색하여 최종 상위 결과 반환
        
        Args:
            query (str): 검색 쿼리
            top_n_per_collection (int): 각 컬렉션에서 가져올 결과 수
            final_top_n (int): 최종 반환할 결과 수
            alpha (float): BM25와 벡터 검색 결합 비율
        
        Returns:
            list: 최종 검색 결과 리스트
        """
        collections = ["1_Embryology", "2_Osteology", "3_Syndesmology", "4_Myology", 
                    "5_Angiology", "6_The_Arteries", "7_The_Veins", "8_The_Lymphatic_System", 
                    "9_Neurology", "10_The_Organs_of_the_Senses_and_the_Common_Integument", 
                    "11_Splanchnology", "12_Surface_Anatomy_and_Surface_Markings"]
        
        all_results = []
        original_collection = getattr(self.vector_db, 'collection_name', None)
        
        # 쿼리 임베딩 생성 (한 번만)
        if not isinstance(query, str):
            query = str(query)
        
        # 구두점으로 쿼리 분할하여 평균 임베딩 생성
        query_parts = re.split(r'[.!?;]', query)
        query_parts = [part.strip() for part in query_parts if part.strip()]
        
        if query_parts:
            part_embeddings = self.embedding_model.encode(query_parts)
            query_embedding = np.mean(part_embeddings, axis=0)
        else:
            query_embedding = self.embedding_model.encode([query])[0]
        
        # 각 컬렉션에서 검색
        for collection_name in collections:
            try:
                # 컬렉션 변경
                self.vector_db.change_collection(collection_name)
                
                # 해당 컬렉션에서 검색
                db_results = self.vector_db.search_with_embedding(
                    embedding=query_embedding.tolist(), 
                    top_n=top_n_per_collection
                )
                
                # 결과에 컬렉션 정보 추가
                for result in db_results:
                    result['collection'] = collection_name
                    result['combined_score'] = 1.0 - result['distance']  # 거리를 유사도로 변환
                    all_results.append(result)
                    
            except Exception as e:
                print(f"컬렉션 {collection_name} 검색 중 오류: {e}")
                continue
        
        # 원래 컬렉션으로 복원
        if original_collection:
            try:
                self.vector_db.change_collection(original_collection)
            except:
                pass
        
        # 점수 기준으로 정렬하여 상위 결과 반환
        all_results.sort(key=lambda x: x['combined_score'], reverse=True)
        final_results = all_results[:final_top_n]
        
        # 기존 형식에 맞게 변환
        formatted_results = []
        for result in final_results:
            formatted_results.append({
                "chunk": result.get('chunk', result.get('document', '')),
                "score": result['combined_score'],
                "bm25_score": 0.0,  # 벡터 검색만 사용
                "vector_score": result['combined_score'],
                "memory_score": result['combined_score'],
                "index": result.get('index', 0),
                "page": result.get('page', 1),
                "collection": result['collection']
            })
        
        return formatted_results
    
    def load_document(self, file_path):
        """문서 로드 및 청크 분할"""
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()
        
        # 문서를 청크로 분할
        self.chunks, self.chunk_metadata = self._split_into_chunks_with_metadata(content)
        
        # BM25 인덱스 생성
        tokenized_chunks = [self._simple_tokenize(chunk) for chunk in self.chunks]
        self.bm25_index = BM25Okapi(tokenized_chunks)
        
        # 벡터 임베딩 생성 및 인메모리 인덱스 구축
        self.vector_index = self.embedding_model.encode(self.chunks)
        
        # ChromaDB에 문서 추가
        # 문서 ID 생성
        doc_name = os.path.basename(file_path)
        doc_ids = [f"{doc_name}_{i}" for i in range(len(self.chunks))]
        
        # 메타데이터에 페이지 정보 포함
        metadatas = [{"page": meta["page"], "source": doc_name, "chunk_index": i} 
                    for i, meta in enumerate(self.chunk_metadata)]
        
        # 벡터 DB에 문서 추가
        self.vector_db.add_documents(
            documents=self.chunks,
            metadatas=metadatas,
            ids=doc_ids
        )
        
        return len(self.chunks)
    
    def search(self, query, top_n=5, alpha=0.5, use_vector_db=True):
        """
        하이브리드 검색 수행
        
        Args:
            query (str): 검색 쿼리
            top_n (int): 반환할 결과 수
            alpha (float): BM25와 벡터 검색 결과 결합 비율 (0에 가까울수록 벡터 검색 중시)
            use_vector_db (bool): ChromaDB 벡터 DB 사용 여부
        
        Returns:
            list: 검색 결과 리스트
        """
        if self.chunks is None or self.bm25_index is None:
            raise ValueError("문서가 로드되지 않았습니다. load_document()를 먼저 호출하세요.")
        
        if not isinstance(query, str) :
            query = str(query)
            
        # BM25 검색 수행
        bm25_scores = self.bm25_index.get_scores(self._simple_tokenize(query))
        
        # 두 검색 방식 모두 수행 (디버그 모드)
        query_embedding = self.embedding_model.encode([query])[0]
        memory_scores = cosine_similarity([query_embedding], self.vector_index)[0]
        
        if use_vector_db:
            # ChromaDB를 사용한 벡터 검색
            # db_results = self.vector_db.search(query, top_n=len(self.chunks))
            db_results = self.vector_db.search_with_embedding(
                embedding=query_embedding.tolist(), 
                top_n=len(self.chunks)
            )
            # 결과를 벡터 점수로 변환
            vector_scores = np.zeros(len(self.chunks))
            for res in db_results:
                # 문서 ID에서 인덱스 추출
                chunk_id = res["id"]
                if "_" in chunk_id:
                    try:
                        chunk_index = int(chunk_id.split("_")[-1])
                        if chunk_index < len(self.chunks):
                            # 거리를 유사도로 변환 (코사인 거리는 1 - 코사인 유사도)
                            if res["distance"] is not None:
                                # ChromaDB가 코사인 거리를 사용하는 경우
                                similarity = 1.0 - res["distance"]
                                vector_scores[chunk_index] = similarity
                    except ValueError:
                        continue
            
            # 디버그 모드에서 점수 비교
            if self.debug_mode:
                print("\n벡터 검색 점수 비교:")
                for i in range(min(5, len(self.chunks))):
                    print(f"Chunk {i}: Memory={memory_scores[i]:.4f}, ChromaDB={vector_scores[i]:.4f}, 차이={memory_scores[i]-vector_scores[i]:.4f}")
        else:
            # 메모리 내 벡터 검색 수행 (기존 방식)
            vector_scores = memory_scores
        
        # 검색 결과 결합
        combined_scores = self._combine_scores(bm25_scores, vector_scores, alpha)
        
        # 상위 N개 결과 반환
        top_indices = np.argsort(combined_scores)[-top_n:][::-1]
        results = [
            {
                "chunk": self.chunks[i],
                "score": combined_scores[i],
                "bm25_score": bm25_scores[i],
                "vector_score": vector_scores[i],
                "memory_score": memory_scores[i],  # 디버깅용
                "index": i,
                "page": self.chunk_metadata[i]["page"]
            }
            for i in top_indices
        ]
        
        return results

    def multi_sentence_hybrid_search(self, query, top_n=5, similarity_threshold=0.7, alpha=0.5, use_vector_db=True):
        """
        가상 문서를 문장 단위로 분할하여 각 문장으로 하이브리드 검색 수행 후 결과 병합
        
        Args:
            query (str): 검색 쿼리
            top_n (int): 반환할 결과 수
            similarity_threshold (float): 하이브리드 점수 임계값 (이 값 이상인 결과만 포함)
            alpha (float): BM25와 벡터 검색 결과 결합 비율 (0에 가까울수록 벡터 검색 중시)
            use_vector_db (bool): ChromaDB 벡터 DB 사용 여부
        
        Returns:
            list: 검색 결과 리스트
        """
        if self.chunks is None or self.bm25_index is None:
            raise ValueError("문서가 로드되지 않았습니다. load_document()를 먼저 호출하세요.")
        
        if not isinstance(query, str):
            query = str(query)
        
        # 구두점으로 쿼리 분할
        query_parts = re.split(r'[.!?;]', query)
        query_parts = [part.strip() for part in query_parts if part.strip()]
        
        if not query_parts:
            # 분할 결과가 없으면 기존 mean_search 사용
            return self.mean_search(query, top_n, alpha=alpha, use_vector_db=use_vector_db)
        
        # 각 문장별 검색 결과를 저장할 리스트
        all_results = []
        
        # 각 문장에 대해 검색 수행
        for part in query_parts:
            # 문장 임베딩 생성
            part_embedding = self.embedding_model.encode([part])[0]
            
            # BM25 검색 수행
            bm25_scores = self.bm25_index.get_scores(self._simple_tokenize(part))
            
            if use_vector_db:
                # ChromaDB를 사용한 벡터 검색
                try:
                    part_results = self.vector_db.search(
                        query_embeddings=[part_embedding.tolist()],
                        n_results=len(self.chunks) // 2  # 각 문장당 검색 결과 수 제한
                    )
                except Exception as e:
                    # search 메서드에 문제가 있으면 search_with_embedding 사용
                    try:
                        part_results = self.vector_db.search_with_embedding(
                            embedding=part_embedding.tolist(),
                            top_n=len(self.chunks) // 2
                        )
                    except Exception as e:
                        print(f"검색 오류 발생: {str(e)}")
                        # 오류 발생 시 빈 결과 사용
                        part_results = []
                
                # 벡터 점수 초기화
                vector_scores = np.zeros(len(self.chunks))
                
                # 결과를 벡터 점수로 변환
                for res in part_results:
                    # 문서 ID에서 인덱스 추출
                    chunk_id = res.get("id", "")
                    chunk_index = -1
                    
                    if "_" in chunk_id:
                        try:
                            chunk_index = int(chunk_id.split("_")[-1])
                        except ValueError:
                            continue
                    
                    if 0 <= chunk_index < len(self.chunks):
                        # 거리를 유사도로 변환 (코사인 거리는 1 - 코사인 유사도)
                        similarity = 1.0 - res.get("distance", 0) if res.get("distance") is not None else 0
                        vector_scores[chunk_index] = similarity
            else:
                # 메모리 내 벡터 검색 수행
                vector_scores = cosine_similarity([part_embedding], self.vector_index)[0]
            
            # 하이브리드 점수 계산
            combined_scores = self._combine_scores(bm25_scores, vector_scores, alpha)
            
            # 임계값 이상인 결과 필터링 및 추가
            for i, score in enumerate(combined_scores):
                if score >= similarity_threshold:
                    all_results.append({
                        "chunk_index": i,
                        "combined_score": score,
                        "bm25_score": bm25_scores[i],
                        "vector_score": vector_scores[i],
                        "query_part": part
                    })
        
        # 결과 병합 및 중복 제거 (같은 청크는 가장 높은 하이브리드 점수 값 유지)
        merged_results = {}
        for result in all_results:
            chunk_index = result["chunk_index"]
            combined_score = result["combined_score"]
            
            if chunk_index in merged_results:
                # 기존 점수보다 높으면 업데이트
                if combined_score > merged_results[chunk_index]["combined_score"]:
                    merged_results[chunk_index] = result
            else:
                merged_results[chunk_index] = result
        
        # 하이브리드 점수 기준 내림차순 정렬 후 상위 N개 선택
        sorted_results = sorted(
            merged_results.values(), 
            key=lambda x: x["combined_score"], 
            reverse=True
        )[:top_n]
        
        # 최종 결과 형식 변환
        final_results = []
        for result in sorted_results:
            chunk_index = result["chunk_index"]
            final_results.append({
                "chunk": self.chunks[chunk_index],
                "score": result["combined_score"],  # 하이브리드 점수
                "bm25_score": result["bm25_score"],
                "vector_score": result["vector_score"],
                "memory_score": 0,  # 추적용
                "index": chunk_index,
                "page": self.chunk_metadata[chunk_index]["page"],
                "query_part": result["query_part"]  # 일치한 쿼리 부분 (디버깅용)
            })
        
        return final_results

    def page_search(self, query, top_n=5, alpha=0.5, use_vector_db=True):
        """
        하이브리드 검색 수행 후 검색된 청크가 포함된 전체 페이지를 반환
        
        Args:
            query (str): 검색 쿼리
            top_n (int): 반환할 결과 수
            alpha (float): BM25와 벡터 검색 결과 결합 비율 (0에 가까울수록 벡터 검색 중시)
            use_vector_db (bool): ChromaDB 벡터 DB 사용 여부
        
        Returns:
            list: 검색 결과 리스트 (각 결과는 페이지 전체 내용 포함)
        """
        # 기존 mean_search 함수로 청크 검색
        chunk_results = self.mean_search(query, top_n, alpha, use_vector_db)
        
        # 검색된 페이지 번호 추출
        found_pages = set(result['page'] for result in chunk_results)
        
        # 각 페이지의 모든 청크 찾기
        page_contents = {}
        for page_num in found_pages:
            # 해당 페이지의 모든 청크 찾기
            page_chunks = []
            for i, metadata in enumerate(self.chunk_metadata):
                if metadata['page'] == page_num:
                    page_chunks.append({
                        "chunk": self.chunks[i],
                        "index": i,
                        "page": page_num
                    })
            
            # 청크 인덱스 순으로 정렬
            page_chunks.sort(key=lambda x: x['index'])
            
            # 페이지 내용 결합
            page_content = "\n\n".join([chunk["chunk"] for chunk in page_chunks])
            page_contents[page_num] = page_content
        
        # 원래 검색 결과의 순서를 유지하면서 페이지 전체 내용으로 대체
        full_page_results = []
        for result in chunk_results:
            page_num = result['page']
            full_page_results.append({
                "chunk": page_contents[page_num],  # 청크 대신 페이지 전체 내용
                "score": result['score'],
                "bm25_score": result['bm25_score'],
                "vector_score": result['vector_score'],
                "memory_score": result['memory_score'],
                "index": result['index'],
                "page": page_num
            })
        
        return full_page_results

    def mean_search(self, query, top_n=5, alpha=0.5, use_vector_db=True):
        """
        하이브리드 검색 수행
        
        Args:
            query (str): 검색 쿼리
            top_n (int): 반환할 결과 수
            alpha (float): BM25와 벡터 검색 결과 결합 비율 (0에 가까울수록 벡터 검색 중시)
            use_vector_db (bool): ChromaDB 벡터 DB 사용 여부
        
        Returns:
            list: 검색 결과 리스트
        """
        if self.chunks is None or self.bm25_index is None:
            raise ValueError("문서가 로드되지 않았습니다. load_document()를 먼저 호출하세요.")
        
        if not isinstance(query, str):
            query = str(query)
        
        # 구두점으로 쿼리 분할
        query_parts = re.split(r'[.!?;]', query)
        query_parts = [part.strip() for part in query_parts if part.strip()]
        
        # 각 부분에 대한 임베딩 생성
        if query_parts:
            part_embeddings = self.embedding_model.encode(query_parts)
            # 임베딩의 평균 계산
            query_embedding = np.mean(part_embeddings, axis=0)
        else:
            # 분할 결과가 없으면 원래 쿼리 사용
            query_embedding = self.embedding_model.encode([query])[0]
            
        # BM25 검색 수행
        bm25_scores = self.bm25_index.get_scores(self._simple_tokenize(query))
        
        # 두 검색 방식 모두 수행 (디버그 모드)
        memory_scores = cosine_similarity([query_embedding], self.vector_index)[0]
        
        if use_vector_db:
            # ChromaDB를 사용한 벡터 검색 - 평균 임베딩 사용
            # query 대신 직접 임베딩 벡터 전달
            # db_results = self.vector_db.search(
            #     query_embeddings=[query_embedding.tolist()], 
            #     n_results=len(self.chunks)
            # )

            db_results = self.vector_db.search_with_embedding(
                embedding=query_embedding.tolist(), 
                top_n=len(self.chunks)
            )
            
            # 결과를 벡터 점수로 변환
            vector_scores = np.zeros(len(self.chunks))
            for res in db_results:
                # 문서 ID에서 인덱스 추출
                chunk_id = res["id"]
                if "_" in chunk_id:
                    try:
                        chunk_index = int(chunk_id.split("_")[-1])
                        if chunk_index < len(self.chunks):
                            # 거리를 유사도로 변환 (코사인 거리는 1 - 코사인 유사도)
                            if res["distance"] is not None:
                                # ChromaDB가 코사인 거리를 사용하는 경우
                                similarity = 1.0 - res["distance"]
                                vector_scores[chunk_index] = similarity
                    except ValueError:
                        continue
            
            # 디버그 모드에서 점수 비교
            if self.debug_mode:
                print("\n벡터 검색 점수 비교:")
                for i in range(min(5, len(self.chunks))):
                    print(f"Chunk {i}: Memory={memory_scores[i]:.4f}, ChromaDB={vector_scores[i]:.4f}, 차이={memory_scores[i]-vector_scores[i]:.4f}")
        else:
            # 메모리 내 벡터 검색 수행 (기존 방식) - 평균 임베딩 사용
            vector_scores = memory_scores
        
        # 검색 결과 결합
        combined_scores = self._combine_scores(bm25_scores, vector_scores, alpha)
        
        # 상위 N개 결과 반환
        top_indices = np.argsort(combined_scores)[-top_n:][::-1]
        results = [
            {
                "chunk": self.chunks[i],
                "score": combined_scores[i],
                "bm25_score": bm25_scores[i],
                "vector_score": vector_scores[i],
                "memory_score": memory_scores[i],  # 디버깅용
                "index": i,
                "page": self.chunk_metadata[i]["page"]
            }
            for i in top_indices
        ]
        
        return results

    def _split_into_chunks_with_metadata(self, text):
        """텍스트를 청크로 분할하고 페이지 정보를 메타데이터로 유지하는 함수"""
        chunks = []
        chunk_metadata = []
        
        # 페이지 패턴 정규식 (####으로 시작하는 페이지 헤더)
        page_pattern = re.compile(r'####\s*Page (\d+)')
        
        # 텍스트를 줄 단위로 처리
        lines = text.split('.')
        current_page = "unknown"
        current_chunk = ""
        
        for line in lines:
            # 페이지 헤더 확인
            page_match = page_pattern.match(line)
            
            if page_match:
                # 새 페이지 시작
                # 현재 청크가 있으면 저장
                if current_chunk.strip():
                    chunks.append(current_chunk.strip())
                    chunk_metadata.append({"page": current_page})
                    current_chunk = ""
                
                # 새 페이지 번호 설정
                current_page = page_match.group(1)
                continue
            
            # 현재 청크에 라인 추가
            current_chunk += line + "\n"
            
            # 청크 크기 확인
            if len(current_chunk) >= self.chunk_size:
                chunks.append(current_chunk.strip())
                chunk_metadata.append({"page": current_page})
                current_chunk = ""  # 새 청크 시작
        
        # 마지막 청크 처리
        if current_chunk.strip():
            chunks.append(current_chunk.strip())
            chunk_metadata.append({"page": current_page})
        
        # 너무 작은 청크 결합 (메타데이터 유지)
        i = 0
        while i < len(chunks) - 1:
            if len(chunks[i]) + len(chunks[i+1]) < self.chunk_size:
                # 같은 페이지인 경우에만 결합
                if chunk_metadata[i]["page"] == chunk_metadata[i+1]["page"]:
                    chunks[i] = chunks[i] + "\n\n" + chunks[i+1]
                    chunks.pop(i+1)
                    chunk_metadata.pop(i+1)
                else:
                    i += 1
            else:
                i += 1
        
        return chunks, chunk_metadata
    
    def _simple_tokenize(self, text):
        """텍스트를 간단히 토크나이징하는 함수"""
        if not isinstance(text, str):
            text = str(text)
        return re.findall(r'\w+', text.lower())
    
    def _combine_scores(self, bm25_scores, vector_scores, alpha=0.5):
        """BM25와 벡터 검색 점수를 결합"""
        # 점수 정규화
        if np.max(bm25_scores) > 0:
            bm25_scores = bm25_scores / np.max(bm25_scores)
        if np.max(vector_scores) > 0:
            vector_scores = vector_scores / np.max(vector_scores)
        
        # 가중 평균 계산
        combined = alpha * bm25_scores + (1 - alpha) * vector_scores
        return combined
    
    def get_chunks_with_page_info(self, indices=None):
        """청크와 페이지 정보 반환"""
        if indices is None:
            return [(chunk, self.chunk_metadata[i]["page"]) for i, chunk in enumerate(self.chunks)]
        else:
            return [(self.chunks[i], self.chunk_metadata[i]["page"]) for i in indices if i < len(self.chunks)]
    
    # 1. 질문이 복합적인지 확인하는 메서드
    def is_complex_question(self, question):
        """
        LLM을 사용하여 질문이 복합적인지 판단하는 메서드
        
        Args:
            question (str): 사용자 질문
            
        Returns:
            bool: 복합적인 질문이면 True, 아니면 False
        """
        prompt = self.complexity_prompt.format(question=question)
        response = generation_loader.generate(prompt)
        
        # '예' 또는 'Yes'가 응답에 포함되어 있으면 복합적인 질문으로 간주
        response = response.lower().strip()
        print(f"###############질문이 복잡한가요? : {response}")
        return '예' in response or 'yes' in response
    
    # 2. 복합적인 질문을 하위 질문으로 분해하는 메서드
    def decompose_question(self, complex_question):
        """
        LLM을 사용하여 복합적인 질문을 여러 개의 하위 질문으로 분해하는 메서드
        
        Args:
            complex_question (str): 복합적인 사용자 질문
            
        Returns:
            list: 하위 질문 목록
        """
        prompt = self.decompose_prompt.format(question=complex_question)
        response = generation_loader.generate(prompt)

        try:
            # JSON 형식으로 반환된 하위 질문 파싱
            # JSON 블록 추출 (```json과 ```로 감싸져 있을 수 있음)
            json_match = re.search(r'```json\s*(.+?)\s*```', response, re.DOTALL)
            if json_match:
                response = json_match.group(1)
            
            # 중괄호 블록 추출
            json_match = re.search(r'(\{.+?\})', response, re.DOTALL)
            if json_match:
                response = json_match.group(1)
                
            sub_questions_data = json.loads(response)
            # 다양한 JSON 형식 처리
            if isinstance(sub_questions_data, list):
                return sub_questions_data
            elif isinstance(sub_questions_data, dict):
                if "questions" in sub_questions_data:
                    return sub_questions_data["questions"]
                elif "subQuestions" in sub_questions_data:
                    return sub_questions_data["subQuestions"]
                else:
                    # 딕셔너리의 값들을 리스트로 반환
                    return list(sub_questions_data.values())
                
        except (json.JSONDecodeError, AttributeError) as e:
            print(f"하위 질문 파싱 오류: {e}")
            print(f"LLM 응답: {response}")
            
            # 파싱 실패 시 줄바꿈을 기준으로 질문 추출 시도
            questions = []
            for line in response.split('\n'):
                line = line.strip()
                if line and ('?' in line or '질문' in line):
                    # 숫자, 점, 괄호 등의 접두어 제거
                    cleaned_line = re.sub(r'^[\d\.\)\-\s]+', '', line).strip()
                    if cleaned_line:
                        questions.append(cleaned_line)
            
            if questions:
                return questions
            # 단일 질문으로 처리
            return [complex_question]

In [ ]:
from model_loader.config import *
from save_utils import *
from translate import *

class CarManualQA:
    def __init__(self, generation_loader, data_folder="./data/split_file", 
                 prompt_path_ko="./prompts/ko/generation/gemma3/generation_prompt2.txt", 
                 prompt_path_en="./prompts/en/generation/gemma3/generation_prompt2.txt", 
                 result_path="./result/5월12일/gemma3",
                 use_vector_db=True, 
                 persist_directory="./chroma_db",
                 collection_name="Gray",
                 language="en"):
        """
        자동차 매뉴얼 Q&A 시스템 초기화
        
        Args:
            generation_loader: 텍스트 생성 모델 로더
            data_folder (str): 분할된 데이터 파일들이 있는 폴더 경로
            prompt_path (str): 프롬프트 템플릿 파일 경로
            result_path (str): 결과를 저장할 경로
            use_vector_db (bool): ChromaDB 벡터 DB 사용 여부
            persist_directory (str): 벡터 DB 저장 경로
            collection_name (str): 벡터 DB 컬렉션 이름
        """
        self.data_folder = data_folder
        self.result_path = result_path
        self.use_vector_db = use_vector_db
        self.language = language
        self.persist_directory = persist_directory
        collection_name = f"{collection_name}"
        
        # 하이브리드 검색기 초기화 - ChromaDB 지원 버전
        self.searcher = HybridSearcher(
            embedding_model=embedding_loader,
            chunk_size=200, 
            chunk_overlap=50,
            persist_directory=persist_directory,
            collection_name=collection_name
        )
        
        self.loader = generation_loader
        self.prompt_path_ko = prompt_path_ko
        self.prompt_path_en = prompt_path_en

        if language=="ko" :
            self.prompt_template = self._load_prompt(prompt_path_ko)
        else :
            self.prompt_template = self._load_prompt(prompt_path_en)
        
        # 카테고리별 파일 매핑
        if language == "en" :
            self.category_to_file = {
                "16": "GrayAnatomy_Formatted.md"
            }
        else :
            self.category_to_file = {
                "16": "full.txt"
            }
        
        try :
            self.translate_en_to_ko = en_to_ko
        except ImportError :
            print("translate.py 모듈을 임포트할 수 없습니다.")
            self.translate_en_to_ko = en_to_ko

        # 이미 처리된 파일 추적
        self.loaded_files = set()
        
        # 전체 파일을 벡터 DB에 미리 로드할지 여부
        self.preload_all = False
        
        # 디버그 모드
        self.debug_mode = True
        
    def preload_documents(self):
        """
        모든 카테고리의 파일을 미리 벡터 DB에 로드
        """
        print("모든 문서를 벡터 DB에 로드 중...")
        for category, filename in self.category_to_file.items():
            file_path = os.path.join(self.data_folder, filename)
            if os.path.exists(file_path):
                print(f"카테고리 {category}: {filename} 로드 중...")
                self._load_document(file_path)
                self.loaded_files.add(file_path)
            else:
                print(f"[경고] 파일을 찾을 수 없습니다: {file_path}")
        
        print(f"총 {len(self.loaded_files)}개 파일이 벡터 DB에 로드되었습니다.")
        self.preload_all = True

    # def _load_document(self, file_path):
    #     """
    #     문서를 로드하여 검색기에 추가
        
    #     Args:
    #         file_path (str): 로드할 파일 경로
            
    #     Returns:
    #         int: 로드된 청크 수
    #     """
    #     # 이미 로드된 파일이면 건너뛰기
    #     if file_path in self.loaded_files and self.preload_all:
    #         if self.debug_mode:
    #             print(f"이미 로드된 파일입니다: {file_path}")
    #         return 0
        
    #     # 파일 로드 및 벡터 DB에 추가
    #     num_chunks = self.searcher.load_document(file_path)
        
    #     # 파일 추적 목록에 추가
    #     self.loaded_files.add(file_path)
        
    #     if self.debug_mode:
    #         print(f"파일 로드 완료: {file_path} ({num_chunks}개 청크)")
        
    #     return num_chunks

    def _load_document(self, file_path):
        """문서 로드 최적화 버전"""
        if isinstance(self.loaded_files, set) :
            self.loaded_files={}

        # 이미 로드된 파일인지 확인
        if file_path in self.loaded_files:
            if self.debug_mode:
                print(f"이미 로드된 파일입니다: {file_path}")
            return self.loaded_files[file_path]
        
        try:
            # 1. 캐시 파일 경로 설정
            cache_dir = os.path.join(os.path.dirname(file_path), '.cache')
            cache_file = os.path.join(cache_dir, os.path.basename(file_path) + '.pickle')
            
            # 2. 캐시 파일이 존재하고 원본보다 최신이면 캐시 로드
            if os.path.exists(cache_file) and os.path.getmtime(cache_file) > os.path.getmtime(file_path):
                with open(cache_file, 'rb') as f:
                    import pickle
                    num_chunks = pickle.load(f)
                    self.loaded_files[file_path] = num_chunks
                    if self.debug_mode:
                        print(f"캐시에서 로드 완료: {file_path}")
                    return num_chunks
            
            # 3. 캐시 없으면 일반 로드 후 캐시 저장
            num_chunks = self.searcher.load_document(file_path)
            self.loaded_files[file_path] = num_chunks
            
            # 캐시 디렉토리 생성
            os.makedirs(cache_dir, exist_ok=True)
            
            # 캐시 파일 저장
            with open(cache_file, 'wb') as f:
                import pickle
                pickle.dump(num_chunks, f)
            
            if self.debug_mode:
                print(f"로드 및 캐시 저장 완료: {file_path}")
            
            return num_chunks
        except Exception as e:
            print(f"문서 로드 중 오류 발생: {e}")
            import traceback
            traceback.print_exc()
            return 0

    def filter_relevant_content(self, query, search_results, threshold=0.7, top_k=5):
        """
        LLM을 사용하여 검색 결과에서 질문과 관련이 있는 내용을 평가하고 
        관련성 점수가 높은 상위 k개 결과만 반환
        
        Args:
            query (str): 사용자 질의
            search_results (list): 검색 결과 목록
            threshold (float): 관련성 점수 임계값 (0.0~1.0)
            top_k (int): 반환할 상위 결과 개수
            
        Returns:
            list: 점수가 높은 상위 k개의 필터링된 관련 정보 목록
        """
        scored_results = []
        
        if self.debug_mode:
            print(f"검색 결과 {len(search_results)}개에 대한 관련성 필터링 시작")
        
        # 각 검색 결과에 대해 관련성 평가
        for result in search_results:
            # 프롬프트 구성 - 관련성 평가용
            relevance_prompt = f"""
    다음은 사용자의 질문입니다:
    "{query}"
    다음은 검색된 텍스트 정보입니다:
    "{result['chunk']}"
    위 텍스트가 사용자 질문에 얼마나 관련이 있는지 평가해주세요.
    평가는 다음과 같이 응답해주세요:
    1. 관련성 점수: 0.0 ~ 1.0 사이의 숫자 (1.0이 가장 관련성 높음)
    2. 이유: 관련성이 높거나 낮은 이유를 간략하게 설명
    응답 형식:
    {{
    "score": 0.0~1.0,
    "reason": "평가 이유"
    }}
    """
            
            relevance_result = self.loader.generate(relevance_prompt)
            
            try:
                # JSON 응답 파싱 (정확한 JSON이 아닐 수 있으므로 예외 처리)
                import re
                import json
                
                # JSON 형식 추출 시도
                json_match = re.search(r'\{.*"score".*:.*,.*"reason".*:.*\}', relevance_result, re.DOTALL)
                if json_match:
                    relevance_data = json.loads(json_match.group(0))
                    score = float(relevance_data.get("score", 0))
                else:
                    # 숫자만 추출 시도
                    score_match = re.search(r'score"?\s*:?\s*(\d+\.\d+|\d+)', relevance_result)
                    score = float(score_match.group(1)) if score_match else 0.0
                
                # 점수와 함께 결과 저장 (임계값 이상인 것만)
                if score >= threshold:
                    if self.debug_mode:
                        print(f"관련성 높음 (점수: {score:.2f}): {result['page']}페이지")
                    # 원본 결과에 점수 정보 추가
                    result_with_score = result.copy()
                    result_with_score['relevance_score'] = score
                    scored_results.append(result_with_score)
                else:
                    if self.debug_mode:
                        print(f"관련성 낮음 (점수: {score:.2f}): {result['page']}페이지")
                    
            except Exception as e:
                # 파싱 실패 시 안전을 위해 낮은 점수로 포함
                if self.debug_mode:
                    print(f"관련성 평가 파싱 오류: {str(e)}")
                result_with_score = result.copy()
                result_with_score['relevance_score'] = 0.1  # 낮은 기본 점수
                scored_results.append(result_with_score)
        
        # 점수를 기준으로 내림차순 정렬
        sorted_results = sorted(scored_results, key=lambda x: x['relevance_score'], reverse=True)
        
        # 상위 k개 결과만 선택
        top_results = sorted_results[:top_k]
        
        if self.debug_mode:
            print(f"필터링 결과: 총 {len(search_results)}개 중 {len(scored_results)}개가 임계값 통과")
            print(f"상위 {min(top_k, len(top_results))}개 결과만 선택됨")
            for i, res in enumerate(top_results):
                print(f"  {i+1}위: 점수 {res['relevance_score']:.2f} - {res['page']}페이지")
        
        # 결과가 없는 경우 원본의 일부라도 반환
        if not top_results and search_results:
            if self.debug_mode:
                print(f"필터링 결과가 없어 원본 상위 결과 {min(top_k, len(search_results))}개 포함")
            top_results = search_results[:min(top_k, len(search_results))]
        
        return top_results
    
    def _format_filtered_results(self, filtered_results):
        """
        필터링된 결과를 문자열로 변환
        
        Args:
            filtered_results (list): 필터링된 검색 결과 리스트
            
        Returns:
            str: 포맷된 문자열
        """
        result_str = ""
        
        for item in filtered_results:
            result_str += f"# 질문: {item['sub_question']}\n\n"
            
            # results가 이미 점수로 정렬되었다고 가정
            for res in item['results']:
                page_info = res['page']
                score_info = ""
                if 'relevance_score' in res:
                    score_info = f" (관련성: {res['relevance_score']:.2f})"
                
                result_str += f"## {page_info}페이지{score_info}\n"
                result_str += f"{res['chunk']}\n\n"
        
        return result_str

    def _generate_hypothetical_document(self, query):
        """
        질문에 대한 가상 문서 생성 (Hyde 기법)
        
        Args:
            query (str): 사용자 질의
            
        Returns:
            str: 생성된 가상 문서
        """
        # Hyde 프롬프트 템플릿
        hyde_prompt = """Write a concise and accurate hypothetical document content for the following question:
Question: {query}
Guidelines:
1. Include only essential information in 200-300 words.
2. Be sure to include important keywords and technical terminology related to the question.
3. Limit to 2-3 paragraphs, with each paragraph addressing a clear topic.
4. Focus on facts and omit unnecessary explanations or examples.
5. Write in an objective style as would appear in a professional reference or textbook.
6. This document should be optimized for vector search. Therefore, include abundant relevant terminology and synonyms while keeping the overall length short.
7. Do not use any markdown syntax in your response.
"""
            
        prompt = hyde_prompt.format(query=query)
        
        # LLM을 사용하여 가상 문서 생성
        if hasattr(self.loader, "tokenizer"):
            tokenizer = self.loader.tokenizer
            model = self.loader.model
            
            # 인풋 토크나이즈
            input_ids = tokenizer.encode(prompt, return_tensors="pt", padding=True, truncation=True).to(model.device)
            attention_mask = (input_ids != tokenizer.pad_token_id).long().to(model.device)
            
            # 텍스트 생성
            output = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_new_tokens=400,
                temperature=0.7,  
                do_sample=True,
                top_p=0.85,
                repetition_penalty=1.2,
                early_stopping=True,
                num_beams=1,  
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id,
            )
            generated_ids = output[0][input_ids.shape[-1]:]
            hypothetical_doc = tokenizer.decode(generated_ids, skip_special_tokens=True).strip()
        else:
            # OpenAI 등 API 기반 모델 사용
            hypothetical_doc = self.loader.generate(prompt)
        
        return hypothetical_doc

    # def search_across_collections(self, query, top_n=5, alpha=0.5):
    #     """모든 컬렉션에서 HyDE 기법을 사용한 검색 후 점수 기준 상위 결과 반환"""
        
    #     collections = ["1_Embryology", "2_Osteology", "3_Syndesmology", "4_Myology", 
    #                 "5_Angiology", "6_The_Arteries", "7_The_Veins", "8_The_Lymphatic_System", 
    #                 "9_Neurology", "10_The_Organs_of_the_Senses_and_the_Common_Integument", 
    #                 "11_Splanchnology", "12_Surface_Anatomy_and_Surface_Markings"]
        
    #     # HyDE 기법 적용
    #     hyde_start = time.time()
    #     hypothetical_doc = self._generate_hypothetical_document(query)
    #     if self.debug_mode:
    #         print(f"가상 문서 : {hypothetical_doc}")
    #     hyde_end = time.time()
    #     if self.debug_mode:
    #         print(f"가상 문서 생성 시간 : {hyde_end - hyde_start}")
        
    #     all_results = []
        
        
    #     for collection_name in collections:
    #         print(f"################{collection_name} 검색 실행################")
    #         # 임시로 컬렉션 변경
    #         original_collection = self.searcher.collection_name
    #         self.searcher.collection_name = collection_name
            
    #         try:
    #             # 각 컬렉션에서 HyDE 문서로 검색
    #             results = self.searcher.mean_search(
    #                 query=hypothetical_doc, 
    #                 top_n=top_n * 2,  # 더 많이 가져와서 선택의 폭을 넓힘
    #                 alpha=alpha,
    #                 use_vector_db=self.use_vector_db
    #             )
                
    #             print(f"{collection_name}에서 {len(results)}개 결과 검색됨")
    #             # 컬렉션 정보 추가
    #             for result in results:
    #                 result['collection'] = collection_name
    #                 all_results.append(result)
                    
    #         except Exception as e:
    #             print(f"컬렉션 {collection_name} 검색 중 오류: {e}")
    #             continue
    #         finally:
    #             # 원래 컬렉션으로 복구
    #             self.searcher.collection_name = original_collection
        
    #     # 점수 기준 정렬 후 상위 top_n개 반환
    #     all_results.sort(key=lambda x: x['score'], reverse=True)
    #     return all_results[:top_n]

    def search_across_collections(self, query, top_n=5, alpha=0.5):
        """모든 컬렉션에서 HyDE 기법을 사용한 검색 후 점수 기준 상위 결과 반환"""
        
        collections = ["1_Embryology", "2_Osteology", "3_Syndesmology", "4_Myology", 
                    "5_Angiology", "6_The_Arteries", "7_The_Veins", "8_The_Lymphatic_System", 
                    "9_Neurology", "10_The_Organs_of_the_Senses_and_the_Common_Integument", 
                    "11_Splanchnology", "12_Surface_Anatomy_and_Surface_Markings"]
        
        # HyDE 기법 적용
        hyde_start = time.time()
        hypothetical_doc = self._generate_hypothetical_document(query)
        if self.debug_mode:
            print(f"가상 문서 : {hypothetical_doc}")
        
        # HyDE 문서가 비어있는지 확인
        if not hypothetical_doc or len(hypothetical_doc.strip()) == 0:
            print("HyDE 문서 생성 실패 - 원본 쿼리 사용")
            hypothetical_doc = query
        
        hyde_end = time.time()
        if self.debug_mode:
            print(f"가상 문서 생성 시간 : {hyde_end - hyde_start}")
        
        all_results = []
        
        # ChromaDB 클라이언트 한 번만 생성
        try:
            client = chromadb.PersistentClient(path=self.searcher.persist_directory)
            available_collections = [col.name for col in client.list_collections()]
            print(f"사용 가능한 컬렉션: {available_collections}")
            
            # 사용 가능한 컬렉션이 없는 경우
            if not available_collections:
                print("사용 가능한 컬렉션이 없습니다!")
                return []
                
        except Exception as e:
            print(f"ChromaDB 클라이언트 연결 실패: {e}")
            return []
        
        for collection_name in collections:
            try:
                # 컬렉션 존재 여부 확인
                if collection_name not in available_collections:
                    print(f"컬렉션 {collection_name}이 존재하지 않습니다. 건너뜁니다.")
                    continue
                    
                # 기존 컬렉션 가져오기
                collection = client.get_collection(
                    name=collection_name,
                    embedding_function=self.searcher.vector_db.embedding_function
                )
                
                # 컬렉션 크기 확인
                collection_count = collection.count()
                print(f"현재 검색 중인 컬렉션: {collection_name} (문서 수: {collection_count})")
                
                if collection_count == 0:
                    print(f"컬렉션 {collection_name}이 비어있습니다.")
                    continue
                
                # 쿼리 임베딩 생성 (HyDE 문서 사용)
                try:
                    query_embedding = self.searcher.embedding_model.encode([hypothetical_doc])[0]
                    print(f"임베딩 생성 완료: 차원 = {len(query_embedding)}")
                except Exception as embed_error:
                    print(f"임베딩 생성 실패: {embed_error}")
                    continue
                
                # ChromaDB에서 벡터 검색
                db_results = collection.query(
                    query_embeddings=[query_embedding.tolist()],
                    n_results=min(top_n * 2, collection_count),
                    include=["documents", "metadatas", "distances"]
                )
                
                print(f"{collection_name}에서 {len(db_results['documents'][0]) if db_results['documents'] else 0}개 결과 검색됨")
                
                # 결과가 실제로 있는지 확인
                if not db_results['documents'] or not db_results['documents'][0]:
                    print(f"컬렉션 {collection_name}에서 검색 결과 없음")
                    continue
                
                # 결과 포맷 변환
                for i in range(len(db_results['documents'][0])):
                    # 메타데이터에서 페이지 정보 추출
                    metadata = db_results['metadatas'][0][i] if db_results['metadatas'] else {}
                    page_info = metadata.get('page', f'Page {i+1}')
                    
                    result = {
                        "chunk": db_results['documents'][0][i],
                        "score": 1.0 - db_results['distances'][0][i] if db_results['distances'] else 0.5,
                        "bm25_score": 0,
                        "vector_score": 1.0 - db_results['distances'][0][i] if db_results['distances'] else 0.5,
                        "memory_score": 1.0 - db_results['distances'][0][i] if db_results['distances'] else 0.5,
                        "index": db_results['ids'][0][i] if db_results['ids'] else f"{collection_name}_{i}",
                        "page": page_info,
                        "collection": collection_name
                    }
                    all_results.append(result)
                
                if db_results['documents'][0]:
                    print(f"첫 번째 결과 페이지: {metadata.get('page', 'page 정보 없음')}")
                    print(f"첫 번째 결과 일부: {db_results['documents'][0][0][:100]}...")
                        
            except Exception as e:
                print(f"컬렉션 {collection_name} 검색 중 오류: {e}")
                import traceback
                traceback.print_exc()
                continue
        
        print(f"총 검색된 결과 수: {len(all_results)}")
        
        # 점수 기준 정렬 후 상위 top_n개 반환
        if all_results:
            all_results.sort(key=lambda x: x['score'], reverse=True)
            
            # 중복 제거
            seen_chunks = set()
            unique_results = []
            for result in all_results:
                chunk_hash = hash(result['chunk'][:100])
                if chunk_hash not in seen_chunks:
                    seen_chunks.add(chunk_hash)
                    unique_results.append(result)
                    if len(unique_results) >= top_n:
                        break
            
            print(f"전체 {len(all_results)}개 결과에서 중복 제거 후 상위 {len(unique_results)}개 선택")
            return unique_results
        else:
            print("검색 결과가 없습니다.")
            return []

    def generate_response(self, query, category, top_n=5, alpha=0.5, target_language=None, idx=1):
        """
        질의에 대한 응답 생성
        
        Args:
            query (str): 사용자 질의
            category (str): 검색할 카테고리
            top_n (int): 검색 결과 수
            alpha (float): BM25와 벡터 검색 가중치 (높을수록 BM25 중시)
            
        Returns:
            dict: 생성된 응답 및 메타데이터
        """
        import time
        try:
            response_language = target_language if target_language else self.language
            source_language = self.language

            ####################### Hyde 기법 #######################
            hyde_start = time.time()
            hypothetical_doc = self._generate_hypothetical_document(query)
            if self.debug_mode:
                print(f"가상 문서 : {hypothetical_doc}")
            hyde_end = time.time()
            if self.debug_mode:
                print(f"가상 문서 생성 시간 : {hyde_end - hyde_start}")

            # search_start = time.time()
            # search_results = self.search_across_collections(query, top_n=top_n, alpha=alpha)
            # search_end = time.time()
            # if self.debug_mode :
            #     print("검색 시간 : ", search_end - search_start)
            ####################################################################################
            # search_results = self.searcher.search( # 검색
            #     query=query, 
            #     top_n=top_n, 
            #     alpha=alpha,
            #     use_vector_db=self.use_vector_db
            #     )   
            ####################################################################################
            # search_results = self.searcher.multi_sentence_hybrid_search( 
            #     query=hypothetical_doc, 
            #     top_n=top_n, 
            #     similarity_threshold=0.7,
            #     alpha=alpha,
            #     use_vector_db=self.use_vector_db
            #     )   
            ####################################################################################
            search_start = time.time()
            
            search_results = self.searcher.multi_collection_search( 
                query=hypothetical_doc, 
                top_n_per_collection=5, 
                alpha=alpha,
                final_top_n=10
                )   
            
            print("##############################다중 컬렉션 검색 결과 : ", search_results)
            
            search_end = time.time()
            if self.debug_mode :
                print("검색 시간 : ", search_end - search_start)
            ####################################################################################
            # search_results = self.searcher.page_search(
            #     query=hypothetical_doc, 
            #     top_n=top_n, 
            #     alpha=alpha,
            #     use_vector_db=self.use_vector_db
            # )

            # context 생성 부분
            context = "\n\n".join([f"#### Page {result['page']}\n{result['chunk']}" for result in search_results])

            prompt = self.prompt_template.format(context=context, query=query)

            generation_start = time.time()
            # 응답 생성 파트
            if hasattr(self.loader, "tokenizer"):
                # Huggingface 모델 사용
                tokenizer = self.loader.tokenizer
                model = self.loader.model
                
                # 인풋 토크나이즈
                input_ids = tokenizer.encode(prompt, return_tensors="pt", padding=True, truncation=True).to(model.device)
                attention_mask = (input_ids != tokenizer.pad_token_id).long().to(model.device)
                
                # 텍스트 생성
                output = model.generate(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    max_new_tokens=400,
                    temperature=0.3,
                    do_sample=False,
                    top_p=0.85,
                    repetition_penalty=1.2,
                    early_stopping=True,
                    num_beams=3,
                    pad_token_id=tokenizer.pad_token_id,
                    eos_token_id=tokenizer.eos_token_id,
                )
                generated_ids = output[0][input_ids.shape[-1]:]
                raw_answer = tokenizer.decode(generated_ids, skip_special_tokens=True).strip()
            else:
                # OpenAI 등 API 기반 모델 사용 (Ollama 포함)
                raw_answer = self.loader.generate(prompt)
            generation_end = time.time()
            if self.debug_mode :
                print("응답 생성 시간 : ", generation_end - generation_start)

            # 응답 후처리
            final_answer = self._extract_answer_content(raw_answer)
            final_answer = self._remove_chinese_characters(final_answer)
            
            translated_answer = None
            if source_language == "en" and response_language == "ko" :
                if self.translate_en_to_ko :
                    try :
                        translate_start = time.time()
                        translated_answer = self.translate_en_to_ko(final_answer)
                        translate_end = time.time()
                        if self.debug_mode :
                            print("영어 -> 한국어 번역 완료, 번역 시간 : ", translate_end - translate_start)
                    except Exception as e :
                        print(f"번역 중 오류 발생 : {e}")
                        translated_answer = f"[번역 오류] {final_answer}"
                else :
                    translated_answer = f"[번역 모듈 없음] {final_answer}"
            # 결과 준비
            response = {
                "답변": raw_answer,
                "후처리": final_answer,
                "문서 일부": context,
                "question_en": query,
                "answer_en": raw_answer,
                # "is_complex": is_complex,
                # "필터링_결과": all_filtered_results
            }
            
            if translated_answer :
                response["번역된 답변"] = translated_answer
                
            
            # 결과 저장
            try:
                # result_path가 없으면 상위 스코프나 기본값으로 설정
                result_path = getattr(self, 'result_path', '../result')
                
                
                # 폴더가 없으면 생성
                os.makedirs(result_path, exist_ok=True)

                # 파일 저장
                if self.debug_mode:
                    print(f"저장 시도: {result_path}")
                save_response_to_file(
                    query=query,
                    answer=response["답변"],
                    # final_answer=response["번역된 답변"],
                    context=response["문서 일부"],
                    folder=result_path,
                    # hypothetical_doc=hypothetical_doc,
                    idx=idx
                    # filtered_results=filtered_results_str
                )
                if self.debug_mode:
                    print(f"저장 완료: {result_path}")
            except Exception as e:
                print(f"결과 저장 중 오류 발생: {e}")
            
            return response
            
        except Exception as e:
            error_msg = f"[생성 오류] {str(e)}"
            print(error_msg)
            import traceback
            traceback.print_exc()
            return {"답변": error_msg, "후처리": error_msg}
    
    def _load_prompt(self, path):
        """프롬프트 템플릿 로드"""
        try:
            with open(path, "r", encoding="utf-8") as f:
                return f.read()
        except Exception as e:
            print(f"[ERROR] 프롬프트 로드 실패: {e}")
            raise
    
    def _extract_answer_content(self, text):
        """응답에서 답변 부분만 추출"""
        pattern = r"(?:<\|?|<|)?\|?answer\|?(?:\|?>|>)?(.*?)(?:<\|?|<|)?\|?endanswer\|?(?:\|?>|>)?"
        match = re.search(pattern, text, re.DOTALL | re.IGNORECASE)
        return match.group(1).strip() if match else text.strip()
    
    def _remove_chinese_characters(self, text):
        """중국어 문자 제거"""
        return re.sub(r'[\u4E00-\u9FFF]', '', text)
    
    def get_collection_stats(self):
        """벡터 DB 컬렉션 통계 조회"""
        if hasattr(self.searcher, 'vector_db') and hasattr(self.searcher.vector_db, 'get_collection_stats'):
            return self.searcher.vector_db.get_collection_stats()
        return {"error": "벡터 DB 통계를 조회할 수 없습니다."}
    
    def toggle_vector_db(self, use_vector_db=None):
        """벡터 DB 사용 여부 전환"""
        if use_vector_db is not None:
            self.use_vector_db = use_vector_db
        else:
            self.use_vector_db = not self.use_vector_db
        
        print(f"벡터 DB 사용 여부: {self.use_vector_db}")
        return self.use_vector_db
    
    def set_debug_mode(self, debug_mode=True):
        """디버그 모드 설정"""
        self.debug_mode = debug_mode
        return self.debug_mode

In [7]:
# vector_db = ChromaVectorDB(
#     embedding_model=embedding_loader,
#     persist_directory="./chroma_db",
#     collection_name="Gray_en"
# )
# vector_db.delete_collection()
# print("기존 컬렉션 삭제 완료")

In [15]:
from model_loader.config import *
import chromadb
import os
import numpy as np
import time

questions = [
    "How many bones make up the adult human skeleton?",
    "What are the four main classes of bones?",
    "Where does segmentation of the fertilized ovum occur?",
    "What is the role of the notochord?",
    "What are the two types of bone marrow and what are their differences?",
    "What is the difference between intramembranous ossification and intracartilaginous ossification in bone formation?",
    "What are the structural components that make up a typical vertebra?",
    "What constitutes the fetal and maternal portions of the placenta?",
    "What major changes are observed in the embryo during the fourth week of development?",
    "How are the neural folds and neural tube formed during human embryonic development?",
    "Where does the abdominal aorta begin and end?",
    "How many valves are in the femoral vein?",
    "What are the main tributaries of the portal vein?",
    "Where do the pulmonary veins begin and where do they end?",
    "How long is the superior vena cava and where does it begin?",
    "What are the main tributaries of the coronary sinus of the heart?",
    "How are the branches of the abdominal aorta classified?",
    "Where does the external jugular vein begin and where does it end?",
    "What are the different types of external cerebral veins?",
    "What are the superficial veins of the lower extremity?",
    "What are the main functions of the mammary glands and where are they located in the human body?",
    "How is the thyroid gland structured and what are its main anatomical relationships?",
    "What are the parathyroid glands and how do they differ from the thyroid gland in structure and function?",
    "Describe the structure and function of the thymus gland and how it changes throughout life.",
    "What is the hypophysis cerebri (pituitary body) and how is it structured?",
    "What is the pineal body, how is it structured, and what is its potential function?",
    "How are the suprarenal glands structured and what are their main relationships to surrounding structures?",
    "Describe the structure and function of the spleen and its relationships to surrounding organs.",
    "How is the surface anatomy of the head and neck organized, particularly regarding the bony landmarks?",
    "What are the surface markings of the abdomen and how are they used to locate underlying structures?"
]
category_list = ["16"] * 30



generation_loader = generation_loader
# result_base_path = "../result"
qa_system = CarManualQA(
        generation_loader=generation_loader,
        data_folder="./data",
        prompt_path_en="./prompts/en/generation/gemma3/generation_prompt3.txt",
        result_path="./result/5월16일/en-gemma3",
        use_vector_db=True,
        persist_directory="./chroma_db",
        collection_name="Gray",  # 기본 컬렉션은 유지하되, 다중 검색에서는 무시됨
        language="en"
    )
    
alpha_values = [round(i * 0.1, 1) for i in range(7, 11)]
for alpha in alpha_values :
    # alpha_result_path = f"./result/5월23일/en-gemma3/HyDE_mean_split/alpha_{alpha}"
    alpha_result_path = f"./result/5월23일"
    os.makedirs(alpha_result_path, exist_ok=True)

    qa_system.result_path = alpha_result_path

    print(f"\n======================Alpha 값 : {alpha}======================")

    for idx, (q, c) in enumerate(zip(questions, category_list)):
        print(f"\n새 쿼리: {q}, 카테고리: {c}")
        
        start_time = time.time()
        chroma_response = qa_system.generate_response(q, c, top_n=5, alpha=alpha, target_language="en", idx=idx+1)
        end_time = time.time()
        
        elapsed_time = end_time - start_time

        print(f"추론시간 : {elapsed_time}")
        print(f"LLM 답변: {chroma_response['후처리']}")

기존 컬렉션 'Gray'을 로드했습니다.

======================Alpha 값 : 0.7======================

새 쿼리: How many bones make up the adult human skeleton?, 카테고리: 16


/opt/conda/envs/sangwon/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:679: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


가상 문서 : 8. Keep the tone informative and factual. Avoid using personal opinions, jokes, colloquialisms, slang, etc.  | Introduction |
The adult human skeleton is composed of numerous interconnected elements that support the body structure, protect vital organs, facilitate movement, and provide attachment points for muscles. Understanding this complex system can help individuals appreciate its importance in maintaining health and wellness throughout life. Here we examine the number of bones making up the human skeletal framework.| Bones of the Adult Human Skeleton |
According to standard biological classification, the human adult skeleton consists of 206 distinct bones. These are organized into four primary categories: axial (core) and appendicular (limb) structures. The axial skeleton includes the skull, vertebral column, ribcage, sternum, sacrum, coccyx, and thoracic cage components; whereas the appendicular skeleton encompasses all limbs' constituents - upper arm, forearm, hand, lowe

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

##############################다중 컬렉션 검색 결과 :  [{'chunk': 'The bones belonging to this class are: the clavicle, humerus, radius, ulna, femur, tibia, fibula,\nmetacarpals, metatarsals, and phalanges\n\n\nShort Bones\n —Where a part of the skeleton is intended for strength and compactness combined with\nlimited movement, it is constructed of a number of short bones, as in the carpu s and tarsus', 'score': 0.6268966197967529, 'bm25_score': 0.0, 'vector_score': 0.6268966197967529, 'memory_score': 0.6268966197967529, 'index': 0, 'page': '37', 'collection': '2_Osteology'}, {'chunk': '1\n\n\nIn the skeleton of the adult there are 206 distinct bones, as follows:— 2\n\n\nAxial\n\nSkeleton\n\n\nVertebral column 26\n\nSkull 22\n\nHyoid bone 1\n\nRibs and sternum 25\n\n\n— 74\n\nAppendicular Upper extremities 64\n\n\nSkeleton\n\n\nUpper extremities 64\n\nLower extremities 62\n\n\n— 126\n\nAuditory ossicles 6\n\n—\n\nTotal 206\n\nThe patellæ are included in this enumeration, but the smaller sesamoid

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

##############################다중 컬렉션 검색 결과 :  [{'chunk': 'Ossification\n —Some bones are preceded by membrane, such as those forming the roof and sides of the\nskull; others, such as the bones of the limbs, are preceded by rods of cartilage\n Hence two kinds of\nossification are described: the intramembranou s and the intracartilaginous', 'score': 0.6255343556404114, 'bm25_score': 0.0, 'vector_score': 0.6255343556404114, 'memory_score': 0.6255343556404114, 'index': 0, 'page': '37', 'collection': '2_Osteology'}, {'chunk': 'By the agency of these cells a thin layer of bony tissue is formed between\nthe perichondrium and the cartilage, by the intramembranous mode of ossification just described\n There are\nthen, in this first stage of ossification, two processes going on simultaneously: in the center of the cartilage\nthe formation of a number of oblong spaces, formed of calcified matrix and containing the withered cartilage\ncells, and on the surface of the cartilage the formation of a l

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

##############################다중 컬렉션 검색 결과 :  [{'chunk': 'The body is like that of a cervical\nvertebra, being broad transversely; its upper surface is concave, and lipped on either side\n The superior\narticular surface s are directed upward and backward; the spinous process is thick, long, and almost\nhorizontal', 'score': 0.5560210347175598, 'bm25_score': 0.0, 'vector_score': 0.5560210347175598, 'memory_score': 0.5560210347175598, 'index': 0, 'page': '37', 'collection': '2_Osteology'}, {'chunk': 'The superior articular processes\nare thin plates of bone projecting upward from the junctions of the pedicles and laminæ; their articular facets\nare practically flat, and are directed backward and a little lateralward and upward', 'score': 0.5358022451400757, 'bm25_score': 0.0, 'vector_score': 0.5358022451400757, 'memory_score': 0.5358022451400757, 'index': 1, 'page': '37', 'collection': '2_Osteology'}, {'chunk': 'In front, it is in\nrelation with the Psoas major; behind, with the muscles

: 

# 카테고리별 분할(ChromaDB)

In [ ]:
collections = ["1_Embryology", "2_Osteology", "3_Syndesmology", "4_Myology", "5_Angiology", "6_The_Arteries", "7_The_Veins", "8_The_Lymphatic_System", "9_Neurology", "10_The_Organs_of_the_Senses_and_the_Common_Integument", "11_Splanchnology", "12_Surface_Anatomy_and_Surface_Markings"]

for i, collection_name in enumerate(collections) :
    file_path = f"./data/split_file/anatomy/{collection_name}.md"

    qa_system = CarManualQA(
        generation_loader=generation_loader,
        data_folder="./data",
        prompt_path_en="./prompts/en/generation/gemma3/generation_prompt3.txt",
        result_path="./result/5월22일/en-gemma3",
        use_vector_db=True,
        persist_directory="./chroma_db",
        collection_name=collection_name,
        language="en"
    )

    num_chunks = qa_system._load_document(file_path)
    print(f"{collection_name} : {num_chunks} chunks 로드 완료")
    

컬렉션 로드 중 오류 발생: Collection 1_Embryology does not exist.
새 컬렉션 '1_Embryology'을 생성합니다.
로드 및 캐시 저장 완료: ./data/split_file/anatomy/1_Embryology.md
1_Embryology : 360 chunks 로드 완료
컬렉션 로드 중 오류 발생: Collection 2_Osteology does not exist.
새 컬렉션 '2_Osteology'을 생성합니다.
로드 및 캐시 저장 완료: ./data/split_file/anatomy/2_Osteology.md
2_Osteology : 1912 chunks 로드 완료
컬렉션 로드 중 오류 발생: Collection 3_Syndesmology does not exist.
새 컬렉션 '3_Syndesmology'을 생성합니다.
로드 및 캐시 저장 완료: ./data/split_file/anatomy/3_Syndesmology.md
3_Syndesmology : 902 chunks 로드 완료
컬렉션 로드 중 오류 발생: Collection 4_Myology does not exist.
새 컬렉션 '4_Myology'을 생성합니다.
로드 및 캐시 저장 완료: ./data/split_file/anatomy/4_Myology.md
4_Myology : 1416 chunks 로드 완료
컬렉션 로드 중 오류 발생: Collection 5_Angiology does not exist.
새 컬렉션 '5_Angiology'을 생성합니다.
로드 및 캐시 저장 완료: ./data/split_file/anatomy/5_Angiology.md
5_Angiology : 458 chunks 로드 완료
컬렉션 로드 중 오류 발생: Collection 6_The_Arteries does not exist.
새 컬렉션 '6_The_Arteries'을 생성합니다.
로드 및 캐시 저장 완료: ./data/split_file/anatomy/6_The_Arte